In [ ]:
import socket
import threading
from PIL import Image as PIL_Image # 導入 PIL.Image
import io # 導入 io 模組來處理字節流

# 新增一個全局變量來存儲接收到的圖像
received_image = None
image_lock = threading.Lock() # 用於同步對 received_image 的訪問

class Carame_Accept_Object:
    def __init__(self):
        self.host = '0.0.0.0'  # 監聽所有可用的接口
        self.port = 8001       # 端口號
        self.server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server.bind((self.host, self.port))
        self.server.listen(5)
        print(f"服務器正在監聽 {self.host}:{self.port}")

def RT_Image(camera, client, D_addr):
    global received_image
    print(f"客戶端連接: {D_addr}")
    try:
        while True:
            # 接收圖像大小 (假設為4字節)
            data_len_bytes = client.recv(4)
            if not data_len_bytes:
                break
            data_len = int.from_bytes(data_len_bytes, 'big')

            # 接收圖像數據
            img_data = b''
            while len(img_data) < data_len:
                packet = client.recv(data_len - len(img_data))
                if not packet:
                    break
                img_data += packet
            
            if len(img_data) == data_len:
                try:
                    # 將字節數據轉換為 PIL 圖像
                    # 假設客戶端發送的是 JPEG 或 PNG 格式的圖像字節流
                    img_pil = PIL_Image.open(io.BytesIO(img_data))
                    
                    with image_lock:
                        received_image = img_pil
                    print(f"收到來自 {D_addr} 的圖像，大小: {img_pil.size}")
                except Exception as img_e:
                    print(f"無法解碼來自 {D_addr} 的圖像數據或格式錯誤: {img_e}")
            else:
                print(f"接收到的數據長度不匹配，預期: {data_len}, 實際: {len(img_data)}")
                break

    except Exception as e:
        print(f"處理客戶端 {D_addr} 時發生錯誤: {e}")
    finally:
        client.close()
        print(f"客戶端 {D_addr} 斷開連接")

if __name__ == '__main__':
    camera = Carame_Accept_Object()
    while (1):
        client, D_addr = camera.server.accept()
        clientThread = threading.Thread(None, target=RT_Image, args=(camera, client, D_addr,))
        clientThread.start()

服務器正在監聽 0.0.0.0:8001
客戶端連接: ('192.168.2.1', 52978)
收到來自 ('192.168.2.1', 52978) 的圖像，大小: (2888, 3713)
客戶端連接: ('192.168.2.1', 52987)
收到來自 ('192.168.2.1', 52987) 的圖像，大小: (480, 360)


In [ ]:
img_pil